In [127]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [128]:
df = pd.read_csv('../data/process_v1/process_data_6.csv')
df = df[df['city'] == 0]

In [129]:
df['is_street_house'] = (df['accessibility'] == 2).astype(np.int32)

In [130]:
df['landSize_ratio'] = df['landSize'] / df['acreage']

In [131]:
mean_land_size_df = df.groupby('administrative_genre')['landSize'].mean().reset_index().rename(columns = {'landSize': 'meanLandSize'})

df = df.merge(mean_land_size_df, how='left', on = 'administrative_genre')

df['landSize_ratio_with_administrative_genre'] = df['landSize'] / df['meanLandSize']
df['acreage_ratio_with_meanLandSize'] = df['acreage'] / df['meanLandSize']


del df['meanLandSize']

def partition_for_alley_width(alley_width):
    if math.isnan(alley_width):
        return np.nan
    if alley_width <= 2.5:
        return 1
    if alley_width <= 5:
        return 2
    if alley_width <= 7:
        return 3
    if alley_width <= 8:
        return 4

In [132]:
{'fitThreeCars': 0,
 'fitOneCarAndOneMotorbike': 1,
 'notInTheAlley': 2,
 'parkCar': 3,
 'theBottleNeckPoint': 4,
 'fitTwoCars': 5,
 'narrorRoad': 6}

{'fitThreeCars': 0,
 'fitOneCarAndOneMotorbike': 1,
 'notInTheAlley': 2,
 'parkCar': 3,
 'theBottleNeckPoint': 4,
 'fitTwoCars': 5,
 'narrorRoad': 6}

In [133]:
s1 = u'ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚÝàáâãèéêìíòóôõùúýĂăĐđĨĩŨũƠơƯưẠạẢảẤấẦầẨẩẪẫẬậẮắẰằẲẳẴẵẶặẸẹẺẻẼẽẾếỀềỂểỄễỆệỈỉỊịỌọỎỏỐốỒồỔổỖỗỘộỚớỜờỞởỠỡỢợỤụỦủỨứỪừỬửỮữỰựỲỳỴỵỶỷỸỹ'
s0 = u'AAAAEEEIIOOOOUUYaaaaeeeiioooouuyAaDdIiUuOoUuAaAaAaAaAaAaAaAaAaAaAaAaEeEeEeEeEeEeEeEeIiIiOoOoOoOoOoOoOoOoOoOoOoOoUuUuUuUuUuUuUuYyYyYyYy'
def remove_accents(input_str):
	s = ''
	for c in input_str:
		if c in s1:
			s += s0[s1.index(c)]
		else:
			s += c
	return s

full_cols = df.columns.tolist()
format_full_cols = [remove_accents(col) for col in full_cols]

df = df.rename(columns = dict(zip(full_cols, format_full_cols)))

In [134]:
cat_cols = ['is_street_house', 'ward', 'street', 'district', 'typeOfRealEstate', 'houseDirection', 'accessibility', 'certificateOfLandUseRight'] + [c for c in df.columns.tolist() if ('_district' in c or '_ward' in c or '_street' in c) and 'distance' not in c and 'num' not in c]
target_cols = ['target']
remove_cols = ['price', 'description', 'city', 'endWidth'] + [c for c in df.columns if 'distance_hcm' in c]
time_cols = ['time']
num_cols = [c for c in df.columns.tolist() if c not in (cat_cols + target_cols + remove_cols + time_cols)]

cat_cols = list(set(cat_cols))
target_cols = list(set(target_cols))
remove_cols = list(set(remove_cols))
num_cols = list(set(num_cols))

print(len(cat_cols), len(target_cols), len(remove_cols), len(num_cols))

35 1 53 148


In [135]:
df.columns.tolist()

['numberOfFloors',
 'numberOfBathRooms',
 'numberOfBedRooms',
 'certificateOfLandUseRight',
 'ward',
 'street',
 'district',
 'city',
 'lat',
 'lon',
 'typeOfRealEstate',
 'frontWidth',
 'endWidth',
 'facade',
 'houseDirection',
 'landSize',
 'price',
 'accessibility',
 'description',
 'time',
 'w',
 'h',
 'num_of_university_in_500m_radius',
 'num_of_fuel_in_500m_radius',
 'num_of_cafe_in_500m_radius',
 'num_of_parking_in_500m_radius',
 'num_of_parking_entrance_in_500m_radius',
 'num_of_fast_food_in_500m_radius',
 'num_of_marketplace_in_500m_radius',
 'num_of_restaurant_in_500m_radius',
 'num_of_hospital_in_500m_radius',
 'num_of_school_in_500m_radius',
 'num_of_kindergarten_in_500m_radius',
 'num_of_townhall - community_centre_in_500m_radius',
 'num_of_police_in_500m_radius',
 'num_of_place_of_worship_in_500m_radius',
 'num_of_bank_in_500m_radius',
 'num_of_atm_in_500m_radius',
 'num_of_university_in_1000m_radius',
 'num_of_fuel_in_1000m_radius',
 'num_of_cafe_in_1000m_radius',
 'num_

In [136]:
cat_cols

['certificateOfLandUseRight',
 'nearest_7_street',
 'nearest_1_street',
 'nearest_4_street',
 'nearest_5_district',
 'nearest_2_district',
 'nearest_3_ward',
 'nearest_6_ward',
 'typeOfRealEstate',
 'nearest_5_street',
 'nearest_3_street',
 'nearest_1_ward',
 'nearest_0_ward',
 'nearest_7_ward',
 'ward',
 'is_street_house',
 'houseDirection',
 'nearest_1_district',
 'accessibility',
 'nearest_8_street',
 'nearest_0_district',
 'street',
 'nearest_7_district',
 'nearest_4_ward',
 'nearest_8_district',
 'nearest_2_street',
 'nearest_5_ward',
 'nearest_0_street',
 'nearest_3_district',
 'nearest_2_ward',
 'district',
 'nearest_4_district',
 'nearest_6_street',
 'nearest_6_district',
 'nearest_8_ward']

In [137]:
num_cols

['num_of_parking_in_500m_radius',
 'num_of_restaurant_in_2000m_radius',
 'distance_nearest_1',
 'num_of_fuel_in_500m_radius',
 'num_of_university_in_2000m_radius',
 'distance_hn_SC VivoCity Shopping Center',
 'num_of_bank_in_2000m_radius',
 'distance_hn_Cong vien 23-9',
 'num_of_parking_entrance_in_500m_radius',
 'distance_hn_Cong vien Gia Dinh',
 'num_of_restaurant_in_1000m_radius',
 'numberOfBathRooms',
 'nearest_8_lon',
 'distance_hn_Ben Van Don',
 'nearest_5_lat',
 'num_of_hospital_in_1000m_radius',
 'num_of_townhall - community_centre_in_1000m_radius',
 'nearest_7_lat',
 'num_of_police_in_500m_radius',
 'num_of_fast_food_in_2000m_radius',
 'distance_hn_Kinh Duong Vuong',
 'num_of_police_in_1000m_radius',
 'district_lon',
 'distance_hn_Duong Nam Ky Khoi Nghia',
 'num_of_police_in_2000m_radius',
 'distance_district',
 'distance_hn_Saigon Garden',
 'nearest_2_lon',
 'w',
 'distance_hn_Duong Mai Chi Tho',
 'num_of_kindergarten_in_500m_radius',
 'nearest_6_lat',
 'distance_hn_Duong Ngu

In [138]:
print(len(num_cols))

148


In [139]:
df[cat_cols] = df[cat_cols].fillna(100)
df[cat_cols] = df[cat_cols].astype(np.int32)

In [140]:
df[num_cols]

,num_of_parking_in_500m_radius,num_of_restaurant_in_2000m_radius,distance_nearest_1,num_of_fuel_in_500m_radius,num_of_university_in_2000m_radius,distance_hn_SC VivoCity Shopping Center,num_of_bank_in_2000m_radius,distance_hn_Cong vien 23-9,num_of_parking_entrance_in_500m_radius,distance_hn_Cong vien Gia Dinh,...,distance_hn_Le Van Sy,nearest_4_lon,distance_nearest_4,nearest_0_lon,num_of_marketplace_in_500m_radius,distance_hn_Cong truong Lam Son,distance_hn_Duong Nguyen Thi Dinh,distance_hn_Cong vien Tho Trang,distance_hn_Cong vien ca Koi Rin Rin Park,num_of_restaurant_in_500m_radius
0,0,4,6.345885,0,2,9.613846,3,9.348445,0,9.143510,...,13.985360,NaN,NaN,106.742199,0,9.231474,9.234584,9.375742,9.746764,0
1,2,71,5.587561,0,5,6.460449,12,8.648254,0,9.273441,...,13.997648,106.709831,6.319727,106.712030,0,8.732156,9.027424,9.113070,9.943502,5
2,0,64,5.480425,1,5,7.531876,18,8.246297,0,9.064318,...,13.995683,106.716766,5.716501,106.713926,1,8.285034,8.817685,8.921404,9.866684,1
3,4,45,5.118292,1,4,9.301450,38,8.662036,0,7.884842,...,13.989567,106.650877,5.485793,106.648131,1,8.734114,9.514311,7.649549,9.180683,3
4,1,370,4.875554,1,20,8.936273,126,7.838244,0,7.988425,...,13.991002,106.693809,5.327174,106.691809,2,7.543084,9.078222,8.107241,9.550837,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49047,0,2,6.798738,0,0,9.958589,0,9.677173,0,9.356228,...,13.982751,106.589628,7.265272,106.577800,0,9.692566,10.039333,9.416338,6.328449,0
49048,0,7,6.151324,3,7,9.859584,2,9.531314,0,9.098179,...,13.982633,106.622825,6.211064,106.620903,2,9.525969,9.889534,9.255996,8.331347,1
49049,3,9,6.564167,0,11,9.689969,8,9.574764,0,9.540917,...,13.986806,106.793884,6.675969,106.791773,0,9.474063,9.125928,9.675056,10.030793,0
49050,1,118,4.716857,4,10,8.552219,65,7.833259,0,8.765463,...,13.994447,106.674040,5.487342,106.676583,1,8.266718,9.263548,8.335620,9.659161,6


In [141]:
df = df[cat_cols + num_cols + time_cols + target_cols]

In [142]:
df = df[df[target_cols[0]] != np.inf]
df = df[df[target_cols[0]] != -np.inf]

In [143]:
df.shape

(49052, 185)

In [144]:
df = df[df[target_cols[0]] >= 10]
df = df[df[target_cols[0]] < 100 * 10]
# df = pd.concat([df[df['w'] < 50], df[df['w'].isnull()]])
# df = pd.concat([df[df['h'] < 50], df[df['w'].isnull()]])

In [145]:
for col in num_cols:
    df[col] = df[col].fillna(df[col].mean())

In [146]:
for c in df.columns.tolist():
    if df[c].isnull().sum():
        print(c)
        del df[c]

In [147]:
df.to_csv('../data/process_v1/process_data_7_hcm.csv', index = False)

In [148]:
num_cols

['num_of_parking_in_500m_radius',
 'num_of_restaurant_in_2000m_radius',
 'distance_nearest_1',
 'num_of_fuel_in_500m_radius',
 'num_of_university_in_2000m_radius',
 'distance_hn_SC VivoCity Shopping Center',
 'num_of_bank_in_2000m_radius',
 'distance_hn_Cong vien 23-9',
 'num_of_parking_entrance_in_500m_radius',
 'distance_hn_Cong vien Gia Dinh',
 'num_of_restaurant_in_1000m_radius',
 'numberOfBathRooms',
 'nearest_8_lon',
 'distance_hn_Ben Van Don',
 'nearest_5_lat',
 'num_of_hospital_in_1000m_radius',
 'num_of_townhall - community_centre_in_1000m_radius',
 'nearest_7_lat',
 'num_of_police_in_500m_radius',
 'num_of_fast_food_in_2000m_radius',
 'distance_hn_Kinh Duong Vuong',
 'num_of_police_in_1000m_radius',
 'district_lon',
 'distance_hn_Duong Nam Ky Khoi Nghia',
 'num_of_police_in_2000m_radius',
 'distance_district',
 'distance_hn_Saigon Garden',
 'nearest_2_lon',
 'w',
 'distance_hn_Duong Mai Chi Tho',
 'num_of_kindergarten_in_500m_radius',
 'nearest_6_lat',
 'distance_hn_Duong Ngu

In [149]:
cat_cols

['certificateOfLandUseRight',
 'nearest_7_street',
 'nearest_1_street',
 'nearest_4_street',
 'nearest_5_district',
 'nearest_2_district',
 'nearest_3_ward',
 'nearest_6_ward',
 'typeOfRealEstate',
 'nearest_5_street',
 'nearest_3_street',
 'nearest_1_ward',
 'nearest_0_ward',
 'nearest_7_ward',
 'ward',
 'is_street_house',
 'houseDirection',
 'nearest_1_district',
 'accessibility',
 'nearest_8_street',
 'nearest_0_district',
 'street',
 'nearest_7_district',
 'nearest_4_ward',
 'nearest_8_district',
 'nearest_2_street',
 'nearest_5_ward',
 'nearest_0_street',
 'nearest_3_district',
 'nearest_2_ward',
 'district',
 'nearest_4_district',
 'nearest_6_street',
 'nearest_6_district',
 'nearest_8_ward']